In [190]:
import numpy as np
import datetime as dt 
import csv
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Pre-Processing
Reading in csv files and setting up Dataframes. 

In [206]:
year = 2025
regular_rider_cutoff = 4

# Read and set up data
rides_df = pd.read_csv(f'tbd_stats_{year}.csv')
att = pd.read_csv(f"sat_attendance_{year}.csv")
rider_cols = att.columns[1:]
attendance_df = att.melt(
    id_vars=["date"],
    value_vars=rider_cols,
    var_name="slot",
    value_name="rider"
)
# Set up DFs
attendance_df = attendance_df.dropna(subset=["rider"])
attendance_df = attendance_df[attendance_df["rider"].str.strip() != ""]
attendance_df["rider"] = attendance_df["rider"].str.strip()
attendance_df = attendance_df.drop(columns=["slot"])
# Normalize Dates
rides_df["date"] = pd.to_datetime(rides_df["date"]).dt.normalize()
attendance_df["date"] = pd.to_datetime(attendance_df["date"]).dt.normalize()

/var/folders/6s/2lcz4zcx0515kv6t7ks2qj0h0000gn/T/ipykernel_4362/3472303849.py:20: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/6s/2lcz4zcx0515kv6t7ks2qj0h0000gn/T/ipykernel_4362/3472303849.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [207]:
attendance_df

,date,rider
0,2025-01-04,Anna
1,2025-01-18,Anna
2,2025-02-01,Anna
3,2025-02-15,Anna
4,2025-02-22,Anna
...,...,...
864,2025-08-16,4
901,2025-06-28,Wenbo
905,2025-07-26,Wenbo
908,2025-08-16,5


In [208]:
rides_df

,date,official_ride,start,route,full_route,coffee_shop,num_riders,miles
0,2025-01-04,True,CCT Trailhead,WOD,MVT-WOD,Rare Bird,10,18
1,2025-01-18,True,CCT Trailhead,CCT-RCP,CCT-RCP,Ceremony,8,20
2,2025-02-01,True,CCT Trailhead,CCT-RCP,CCT-RCP,Olivia's Kitchen,14,20
3,2025-02-15,True,Gravelley Point,Alexandria,MVT-Crystal City,Swing's,6,19
4,2025-02-22,True,Navy Yard,ART-MBT,ART-MBT,Vigilante,8,16
5,2025-03-01,True,Gravelley Point,WOD,MVT-WOD,Rare Bird,8,18
6,2025-03-08,True,Gravelley Point,Alexandria,MVT-Crystal City,Hypergoat,9,16
7,2025-03-15,True,Navy Yard,ART-MBT,ART-MBT,Vigilante,14,19
8,2025-03-22,True,CCT Trailhead,CCT-RCP,CCT-RCP,Ceremony,12,20
9,2025-03-29,True,Gateway Park,WOD,Military-WOD,Coffee Republic,14,28


## Attendance Leaderboard

Plot for top riders by attendance leaderboard (min 5 rides this year). This is plotted in both scatter plot and line graph. 
Also plots for shortest hiatus and longest streak among the regular riders. 

In [194]:
attendance_counts = (
    attendance_df.groupby("rider")["date"]
    .nunique()                     # or .count() if multiple entries per date aren't possible
    .reset_index(name="count")
)
attendance_regular = attendance_counts[attendance_counts["count"] > regular_rider_cutoff]
regular_riders = attendance_counts[attendance_counts["count"] > regular_rider_cutoff]["rider"]

In [195]:
def longest_streak(dates):
    dates = sorted(dates)
    if not dates: return 0

    streak = 1
    max_streak = 1

    for i in range(1, len(dates)):
        # 7 Dyas for weekly rides
        if (dates[i] - dates[i-1]).days == 7: streak += 1
        else: streak = 1
        max_streak = max(max_streak, streak)
    return max_streak

streak_df = (
    attendance_df.groupby("rider")["date"]
    .apply(lambda d: longest_streak(list(d)))
    .reset_index(name="longest_streak")
)
streak_df = streak_df[streak_df["rider"].isin(regular_riders)]
streak_df.sort_values("longest_streak", ascending=False, inplace=True)


In [196]:
def longest_hiatus(dates):
    dates = sorted(dates)
    if len(dates) < 2:
        return 0
    gaps = [(dates[i] - dates[i-1]).days for i in range(1, len(dates))]
    return max(gaps)

hiatus_df = (
    attendance_df.groupby("rider")["date"]
    .apply(lambda d: longest_hiatus(list(d)))
    .reset_index(name="longest_hiatus_days")
)
hiatus_df = hiatus_df[hiatus_df["rider"].isin(regular_riders)]
hiatus_df.sort_values("longest_hiatus_days", ascending=False, inplace=True)

In [205]:
fig_top_riders = px.bar(
    attendance_regular, 
    y="rider", x="count",
    title="Rider Attendance Leaderboard", 
    text_auto=True)
fig_top_riders.update_layout(
    title_x=0.5,
    width=600,
    height=800,
    margin=dict(l=5, r=5, t=50, b=5),  # left, right, top, bottom margins
    yaxis=dict(title=None, categoryorder='total ascending'),
    xaxis_title='Number of Rides',
    barmode='stack',
)

fig_top_riders.show()
fig_top_riders.write_image(f'{year}/rider_leaderboard_{year}.png', format='png')
fig_top_riders.write_html(f"{year}/rider_leaderboard_{year}.html")

In [198]:
fig_streak = px.bar(
    streak_df,
    y="rider",
    x="longest_streak",
    title="Longest Attendance Streak per Rider",
    text_auto=True
)
fig_streak.update_layout(
    title_x=0.5,
    width=600,
    height=800,
    margin=dict(l=5, r=5, t=50, b=5),  # left, right, top, bottom margins
    yaxis=dict(title=None, categoryorder='total ascending'),
    xaxis_title='Number of Rides',
    barmode='stack',
)
fig_streak.show()
fig_streak.write_image(f"{year}/attendance_streaks_{year}.png", format="png")
fig_streak.write_html(f"{year}/attendance_streaks_{year}.html")


In [199]:
fig_hiatus = px.bar(
    hiatus_df,
    y="rider",
    x="longest_hiatus_days",
    title="Longest Hiatus Before Comeback",
    text_auto=True
)
fig_hiatus.update_layout(
    title_x=0.5,
    width=600,
    height=800,
    margin=dict(l=5, r=5, t=50, b=5),  # left, right, top, bottom margins
    yaxis=dict(title=None, categoryorder='total ascending'),
    xaxis_title='Number of Rides',
    barmode='stack',
)
fig_hiatus.show()
# fig_hiatus.write_image(f"{year}/hiatus_comeback_{year}.png", format="png")
# fig_hiatus.write_html(f"{year}/hiatus_comeback_{year}.html")

## Coffee Shops Sunburst

In [ ]:
fig_sunburst = px.sunburst(rides_df, path=["route", "coffee_shop"],)
fig_sunburst.update_layout(
    width=450,
    height=800,
    margin=dict(l=5, r=5, t=100, b=5),  # left, right, top, bottom margins
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)
# fig_sunburst.update_traces(textinfo='label')
fig_sunburst.show()
fig_sunburst.write_image(f'{year}/coffee_sunburst_{year}.png', format='png')
fig_sunburst.write_html(f'{year}/coffee_sunburst_{year}.html')

In [ ]:
fig_sunburst = px.sunburst(rides_df, path=["route", "full_route", "coffee_shop"],)
fig_sunburst.update_layout(
    width=450,
    height=800,
    margin=dict(l=5, r=5, t=100, b=5),  # left, right, top, bottom margins
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)
fig_sunburst.show()
fig_sunburst.write_image(f'{year}/ride_sunburst_{year}.png', format='png')
fig_sunburst.write_html(f'{year}/ride_sunburst_{year}.html')

## Ride sizes
Plot for total attendance at rides over time. Includes 4 week rolling average. 

In [226]:
official_rides = rides_df[rides_df["official_ride"] == True]
official_dates = official_rides["date"].unique()
attendance_official = attendance_df[attendance_df["date"].isin(official_dates)]
attendance_counts = attendance_official.groupby("date")["rider"].nunique().reset_index()
attendance_counts = attendance_counts.sort_values("date")
attendance_counts.columns = ['date', 'attendance']

# Rolling average (4 weeks)
attendance_counts["rolling_avg"] = attendance_counts["attendance"].rolling(4, min_periods=1).mean()


In [ ]:
attendance_counts

In [ ]:
rides_df["date"] = pd.to_datetime(rides_df["date"]).dt.normalize()
attendance_df["date"] = pd.to_datetime(attendance_df["date"]).dt.normalize()

In [ ]:
fig_attendance = px.line(
    attendance_counts, x="date", y="attendance", 
    markers=True, 
    title=f"TBD {year} Ride Attendance",
    # line_shape='spline'
)
fig_attendance.update_xaxes(
    fixedrange=False,
    dtick="M1",
    tickformat="%b",
    range=[pd.to_datetime(f'{year}-01-01'),pd.to_datetime(f'{year}-12-31')],
    gridcolor='silver'
)
fig_attendance.update_layout(
    title_x=0.5,
    width=800,
    height=400,
    margin=dict(l=5, r=5, t=50, b=5),  # left, right, top, bottom margins
    yaxis=dict(title=None),
    yaxis_title='Number of Riders',
    plot_bgcolor='#fffcf7',
    paper_bgcolor='#fffcf7',
    # template='simple_white',
)
fig_attendance.update_traces(
    connectgaps=True, 
    line=dict(width=5),
)
# fig_attendance.show()
fig_attendance.write_html(f"{year}/attendance.html")

In [ ]:
fig_attendance.update_layout(
    title_x=0.5,
    width=450,
    height=800,
    margin=dict(l=5, r=5, t=50, b=5),  # left, right, top, bottom margins
    yaxis=dict(title=None),
    yaxis_title='Number of Riders',
    plot_bgcolor='#fffcf7',
    paper_bgcolor='#fffcf7',
    # template='simple_white',
)
fig_attendance.update_traces(
    connectgaps=True, 
    line=dict(width=5),
)
fig_attendance.show()
# fig.write_image(f'{year}/{year}_attendance.png', format='png')

In [ ]:
fig_rolling_avg = go.Figure()

# Scatter points for attendance
fig_rolling_avg.add_trace(go.Scatter(
    x=attendance_counts["date"],
    y=attendance_counts["attendance"],
    mode="markers",
    name="Attendance",
    marker=dict(size=8)
))
# Connected points for rolling average
fig_rolling_avg.add_trace(go.Scatter(
    x=attendance_counts["date"],
    y=attendance_counts["rolling_avg"],
    mode="lines",
    name="4-Ride Rolling Avg",
    line=dict(width=3)
))
fig_rolling_avg.show()
fig_rolling_avg.write_html(f"{year}/attendance_with_rolling_avg_{year}.html")

In [236]:
fig_rolling_avg.update_layout(
    width=450,
    height=800,
    margin=dict(l=5, r=5, t=50, b=5),  # left, right, top, bottom margins
    yaxis=dict(title=None),
    plot_bgcolor='#fffcf7',
    paper_bgcolor='#fffcf7',
    showlegend=False
)
fig_rolling_avg.show()

## Most popular ride by rider count

In [222]:
route_counts = rides_df.groupby("route")["num_riders"].sum().reset_index()
fig_routes_pie = px.pie(
    route_counts,
    names="route",
    values="num_riders",
    title="Route Popularity (Total Riders)"
)
fig_routes_pie.update_layout(
    width=450,
    height=800,
    margin=dict(l=5, r=5, t=100, b=5),  # left, right, top, bottom margins
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)
fig_routes_pie.show()
fig_routes_pie.write_image(f"{year}/route_popularity_pie_{year}.png", format='png')
fig_routes_pie.write_html(f"{year}/route_popularity_pie_{year}.html")


## Numerical Stats

In [ ]:
# Total person-miles
rides_df["person_miles"] = rides_df["num_riders"] * rides_df["miles"]
total_miles = rides_df["person_miles"].sum()

# Median miles
median_rides = rides_df["miles"].median()

# Total number of rides
total_rides = rides_df.shape[0]

# Total unique riders
total_riders = attendance_df["rider"].nunique()

# Riders who appeared only once
one_time_riders_count = (attendance_df.groupby("rider")["date"].nunique() == 1).sum()

# Mean attendance per ride
avg_attendance = attendance_df.groupby("date")["rider"].nunique().mean()

# Median attendance per ride
med_attendance = attendance_df.groupby("date")["rider"].nunique().median()

# Largest single-ride attendance
largest_attendance = rides_df["num_riders"].max()

# Print summary
print(f"Total person-miles ridden: {total_miles}")
print(f"Median ride length: {median_rides}")
print(f"Total rides: {total_rides}")
print(f"Total unique riders: {total_riders}")
print(f"Riders who only appeared once: {one_time_riders_count}")
print(f"Average attendance per ride: {avg_attendance:.2f}")
print(f"Median attendance per ride: {med_attendance:.2f}")
print(f"Largest single-ride attendance: {largest_attendance}")

Total person-miles ridden: 11543
Median ride length: 20.0
Total rides: 44
Total unique riders: 120
Riders who only appeared once: 60
Average attendance per ride: 11.16
Median attendance per ride: 10.00
Largest single-ride attendance: 22


## Long script